<a href="https://colab.research.google.com/github/rodrigogrigo/analise-requisitos/blob/main/MESTRADO_Modelo_LLM_Zero_Shot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U unsloth

from google.colab import drive
drive.mount('/content/drive')

import torch
import pandas as pd
from unsloth import FastLanguageModel

# Lista de modelos a testar: (apelido, nome do modelo Unsloth)
modelos_para_testar = [
    ("gemma_2b", "unsloth/gemma-2b-it-bnb-4bit"),
    ("llama3_8b", "unsloth/llama-3-8b-Instruct-bnb-4bit")
]

# Prompt com instrução clara, exemplos e separador ###
system_role = (
    "Você é um engenheiro de software sênior responsável por estimar o esforço necessário para concluir tarefas, "
    "com base em suas descrições. Utilize a métrica de story points, que representa o esforço relativo.\n\n"
    "Os valores válidos são baseados na sequência de Fibonacci: 1, 2, 3, 5 e 8.\n"
    "- 1: tarefa muito simples\n"
    "- 2 ou 3: tarefa simples ou moderada\n"
    "- 5: tarefa complexa\n"
    "- 8: tarefa de alto esforço\n\n"
    "IMPORTANTE:\n"
    "- Responda com apenas um número: 1, 2, 3, 5 ou 8\n"
    "- Não adicione frases ou símbolos\n"
    "- A resposta deve vir após o marcador ###\n\n"
    "Exemplos:\n"
    "Descrição da tarefa:\nAdicionar autenticação com OAuth2.\n###\n5\n\n"
    "Descrição da tarefa:\nCorrigir erro de digitação em mensagem de boas-vindas.\n###\n1\n\n"
)

prompt_template = """Descrição da tarefa:
{}

###"""

# Caminho para o dataset unificado
csv_path = "/content/drive/MyDrive/projetoMestrado/datasets_all_processados/moodle_unificado_processed.csv"
df = pd.read_csv(csv_path)

# Itera sobre os modelos e executa a inferência para cada um
for apelido_modelo, nome_modelo_unsloth in modelos_para_testar:
    print(f"\n Carregando modelo: {apelido_modelo}")

    # Carregar modelo
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=nome_modelo_unsloth,
        max_seq_length=2048,
        load_in_4bit=True,
        dtype=None,
    )
    FastLanguageModel.for_inference(model)

    resultados = []

    # Loop de inferência
    for _, row in df.iterrows():
        descricao = row["description"]
        issue_id = row["issuekey"]
        storypoint_real = row["storypoint"]

        prompt = prompt_template.format(descricao)
        full_prompt = f"{system_role}\n{prompt}"

        inputs = tokenizer(
            full_prompt,
            return_tensors="pt",
            truncation=True,
            max_length=2048,
            padding=True
        )
        inputs = {k: v.to(device) for k, v in inputs.items()}



        outputs = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs.get("attention_mask", None),
            max_new_tokens=3,
            temperature=0.1,
        )


        result_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
        raw_output = result_text.split("###")[-1].strip()

        resultados.append({
            "dataset": "moodle",
            "id": issue_id,
            "descricao": descricao,
            "storypoint_previsto": raw_output,
            "storypoint_real": storypoint_real
        })

    df_result = pd.DataFrame(resultados)

    # Tentar converter as colunas para int (somente válidos)
    df_result["storypoint_previsto_int"] = pd.to_numeric(df_result["storypoint_previsto"], errors="coerce")
    df_result["storypoint_real_int"] = pd.to_numeric(df_result["storypoint_real"], errors="coerce")

    # Remover valores inválidos (NaN)
    df_validos = df_result.dropna(subset=["storypoint_previsto_int", "storypoint_real_int"])

    # Calcular MAE
    mae = (df_validos["storypoint_previsto_int"] - df_validos["storypoint_real_int"]).abs().mean()

    # Exportar resultados
    output_path = f"/content/drive/MyDrive/projetoMestrado/storypoints_inferidos_{apelido_modelo}.csv"
    df_result.to_csv(output_path, index=False)
    print(f"✅ Resultados exportados: {output_path}")
    print(f"📊 MAE para o modelo {apelido_modelo}: {mae:.2f}")

    # 🔁 Limpar memória GPU antes do próximo modelo
    import gc
    del model
    del tokenizer
    gc.collect()
    torch.cuda.empty_cache()




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

 Carregando modelo: gemma_2b
==((====))==  Unsloth 2025.5.6: Fast Gemma patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
✅ Resultados exportados: /content/drive/MyDrive/projetoMestrado/storypoints_inferidos_gemma_2b.csv
📊 MAE para o modelo gemma_2b: 2.87

 Carregando modelo: llama3_8b
==((====))==  Unsloth 2025.5.6: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 